In [1]:
from purpleair.network import SensorList
import pandas as pd
import numpy as np
import time
import datetime
from datetime import date
from purpleair.sensor import Sensor

In [2]:
p = SensorList()  # Initialized 11,220 sensors!
# Other sensor filters include 'outside', 'useful', 'family', and 'no_child'
df = p.to_dataframe(sensor_filter='all',
                    channel='parent')
# df[df['id'] == 10894]
df[df.index == 10894]['name'].values[0]

Initialized 22,444 sensors!


'Warnow-Chacko'

In [3]:
from purpleair.sensor import Sensor
s = Sensor(2890, parse_location=True)

In [4]:
def get_sensor_data(sensor_id:int):
    name = df[df.index == 10894]['name'].values[0]
    sensor = Sensor(sensor_id, parse_location=True)
    df_sensor = sensor.parent.get_historical(weeks_to_get= 14, start_date = date(2021, 12, 7),
                              thingspeak_field='primary')
    df_sensor.created_at = pd.DatetimeIndex(df_sensor.created_at).tz_convert('US/Central')
    df_sensor['created_at'] = df_sensor['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S').tolist()
    df_sensor['new_date'] = df_sensor.apply(lambda time: each_fifteen(time['created_at']),axis=1)
    df_sensor_date = df_sensor.drop(columns = ['created_at'])
    df_sensor_date.sort_values('new_date', ascending=True)
    df_sensor_date_group = df_sensor_date.groupby('new_date').agg('mean').reset_index()
    df_sensor_date_group['sensor_name'] = name
    return df_sensor_date_group

In [5]:
def export_sensor_data(sensor_id:int):
    name = df[df.index == 10894]['name'].values[0]
    sensor = Sensor(sensor_id, parse_location=True)
    df_sensor = sensor.parent.get_historical(weeks_to_get= 14, start_date = date(2021, 12, 7),
                              thingspeak_field='primary')
    df_sensor.created_at = pd.DatetimeIndex(df_sensor.created_at).tz_convert('US/Central')
    df_sensor['created_at'] = df_sensor['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S').tolist()
    df_sensor['new_date'] = df_sensor.apply(lambda time: each_fifteen(time['created_at']),axis=1)
    df_sensor_date = df_sensor.drop(columns = ['created_at'])
    df_sensor_date.sort_values('new_date', ascending=True)
    df_sensor_date_group = df_sensor_date.groupby('new_date').agg('mean').reset_index()
    df_sensor_date_group['sensor_name'] = name
    df_sensor_date_group.to_feather('export_{}_feather'.format(name))
    return df_sensor_date_group

In [6]:
def each_fifteen(time_str):
    new_time = datetime.datetime.strptime(time_str,'%Y-%m-%d %H:%M:%S')
    new_time = new_time.replace(second = 0)
    if new_time.minute < 8:
        new_time = new_time.replace(minute = 0)
    elif new_time.minute < 23:
        new_time = new_time.replace(minute = 15)
    elif new_time.minute < 38:
        new_time = new_time.replace(minute = 30)
    elif new_time.minute < 53:
        new_time = new_time.replace(minute = 45)
    elif new_time.minute < 60:
        new_time = new_time + datetime.timedelta(hours = 1)
        new_time = new_time.replace(minute = 0)
    return str(new_time)

Warnow-Chacko(South-west): 10894
Urbana(South-east):94415
Urbana on Holmes:107592

In [7]:
data = get_sensor_data(10894)

In [8]:
columns = ['new_date', 'PM1.0 (CF=1) ug/m3', 'PM2.5 (CF=1) ug/m3', 'PM10.0 (CF=1) ug/m3',
          'UptimeMinutes', 'ADC', 'Temperature_F', 'Humidity_%', 'PM2.5 (CF=ATM) ug/m3', 'sensor_name']
df_sensor = pd.DataFrame(columns = columns)

In [9]:
df_sensor.append(data)

,new_date,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,sensor_name
0,2021-08-23 19:00:00,12.050000,18.035000,18.880000,7672.0,-67.500000,94.000000,45.000000,18.035000,Warnow-Chacko
1,2021-08-23 19:15:00,12.226250,17.927500,18.696250,7684.0,-67.250000,94.000000,45.000000,17.927500,Warnow-Chacko
2,2021-08-23 19:30:00,11.734286,17.234286,17.677143,7699.0,-66.714286,92.571429,45.000000,17.234286,Warnow-Chacko
3,2021-08-23 19:45:00,11.942500,17.366250,18.236250,7714.0,-68.125000,93.000000,45.000000,17.366250,Warnow-Chacko
4,2021-08-23 20:00:00,13.081429,19.161429,19.652857,7729.0,-68.714286,93.000000,45.142857,19.161429,Warnow-Chacko
...,...,...,...,...,...,...,...,...,...,...
7565,2021-12-06 17:00:00,0.000000,0.037143,0.100000,208.0,-62.714286,41.000000,25.285714,0.037143,Warnow-Chacko
7566,2021-12-06 17:15:00,0.138750,0.446250,0.623750,223.0,-62.375000,41.000000,25.000000,0.446250,Warnow-Chacko
7567,2021-12-06 17:30:00,0.015714,0.040000,0.152857,238.0,-65.285714,40.857143,25.142857,0.040000,Warnow-Chacko
7568,2021-12-06 17:45:00,0.000000,0.046250,0.046250,253.0,-65.000000,40.000000,24.750000,0.046250,Warnow-Chacko


In [10]:
WarnowChacko = Sensor(10894, parse_location=True)
print(type(WarnowChacko))
Urbana = Sensor(94415, parse_location=True)
print(Urbana)
UrbanaOnHolmes = Sensor(107592, parse_location=True)
print(UrbanaOnHolmes)

<class 'purpleair.sensor.Sensor'>
Sensor 94415 at 1559, South Vine Street, West Urbana, Urbana, Champaign County, Illinois, 61801, United States
Sensor 107592 at 345, East Holmes Street, Urbana, Champaign County, Illinois, 61801, United States


In [11]:
df_WarnowChacko = WarnowChacko.parent.get_historical(weeks_to_get= 14, start_date = date(2021, 12, 7),
                              thingspeak_field='primary')
df_WarnowChacko

,created_at,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3
entry_id,,,,,,,,,
978053,2021-11-30 00:01:19+00:00,2.00,3.57,3.73,18121.0,-68.0,47.0,38.0,3.57
978054,2021-11-30 00:03:19+00:00,2.10,3.81,4.66,18123.0,-66.0,47.0,38.0,3.81
978055,2021-11-30 00:05:19+00:00,2.31,3.81,3.81,18125.0,-66.0,47.0,38.0,3.81
978056,2021-11-30 00:07:19+00:00,2.36,4.11,4.18,18127.0,-67.0,47.0,39.0,4.11
978057,2021-11-30 00:09:19+00:00,1.40,2.96,3.70,18129.0,-66.0,47.0,39.0,2.96
...,...,...,...,...,...,...,...,...,...
931261,2021-08-30 22:36:56+00:00,5.02,7.78,8.00,8651.0,-61.0,87.0,51.0,7.78
931262,2021-08-30 22:38:56+00:00,5.14,6.71,6.71,8653.0,-61.0,87.0,51.0,6.71
931263,2021-08-30 22:40:56+00:00,4.70,6.34,6.34,8655.0,-61.0,87.0,51.0,6.34


In [12]:
df_Urbana = Urbana.parent.get_historical(weeks_to_get= 14, start_date = date(2021, 12, 7),
                              thingspeak_field='primary')
df_Urbana

,created_at,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3
entry_id,,,,,,,,,
194084,2021-11-30 00:00:45+00:00,6.02,6.93,7.28,316.0,-58.0,44.0,45.0,6.93
194085,2021-11-30 00:02:45+00:00,6.86,7.68,7.70,318.0,-58.0,44.0,46.0,7.68
194086,2021-11-30 00:04:45+00:00,6.68,7.43,7.91,320.0,-59.0,44.0,46.0,7.43
194087,2021-11-30 00:06:45+00:00,6.56,7.88,8.27,322.0,-59.0,44.0,46.0,7.88
194088,2021-11-30 00:08:45+00:00,6.79,8.16,8.83,324.0,-59.0,44.0,46.0,8.16
...,...,...,...,...,...,...,...,...,...
128716,2021-08-30 23:50:11+00:00,3.37,4.39,4.59,15779.0,-58.0,86.0,54.0,4.39
128717,2021-08-30 23:52:11+00:00,3.45,4.34,4.34,15781.0,-56.0,86.0,54.0,4.34
128718,2021-08-30 23:54:11+00:00,4.00,5.05,5.57,15783.0,-55.0,85.0,55.0,5.05


In [13]:
df_UrbanaOnHolmes = UrbanaOnHolmes.parent.get_historical(weeks_to_get= 14, start_date = date(2021, 12, 7),
                              thingspeak_field='primary')
df_UrbanaOnHolmes

,created_at,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3
entry_id,,,,,,,,,
94763,2021-12-06 00:11:41+00:00,19.62,31.23,34.21,10.0,-66.0,56.0,75.0,30.29
94764,2021-12-06 00:13:41+00:00,19.92,31.98,34.02,12.0,-66.0,56.0,75.0,30.84
94765,2021-12-06 00:15:41+00:00,19.80,33.96,35.29,14.0,-70.0,56.0,75.0,32.21
94766,2021-12-06 00:17:41+00:00,20.14,31.89,34.39,16.0,-68.0,56.0,75.0,30.89
94767,2021-12-06 00:19:41+00:00,20.43,33.64,35.89,18.0,-68.0,56.0,75.0,31.70
...,...,...,...,...,...,...,...,...,...
30277,2021-08-30 23:50:16+00:00,2.69,4.24,4.24,31567.0,-65.0,83.0,59.0,4.24
30278,2021-08-30 23:52:16+00:00,2.60,4.14,4.14,31569.0,-64.0,83.0,60.0,4.14
30279,2021-08-30 23:54:16+00:00,2.62,4.14,4.30,31571.0,-66.0,83.0,59.0,4.14


In [14]:
df_WarnowChacko.created_at=pd.DatetimeIndex(df_WarnowChacko.created_at).tz_convert('US/Central')
df_WarnowChacko['created_at'] = df_WarnowChacko['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S').tolist()

df_Urbana.created_at=pd.DatetimeIndex(df_Urbana.created_at).tz_convert('US/Central')
df_Urbana['created_at'] = df_Urbana['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S').tolist()

df_UrbanaOnHolmes.created_at=pd.DatetimeIndex(df_UrbanaOnHolmes.created_at).tz_convert('US/Central')
df_UrbanaOnHolmes['created_at'] = df_UrbanaOnHolmes['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S').tolist()

In [15]:
# df_WarnowChacko.apply(each_fifteen, axis = 0)
df_WarnowChacko['new_date'] = df_WarnowChacko.apply(lambda time: each_fifteen(time['created_at']),axis=1)
df_WarnowChacko

,created_at,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,new_date
entry_id,,,,,,,,,,
978053,2021-11-29 18:01:19,2.00,3.57,3.73,18121.0,-68.0,47.0,38.0,3.57,2021-11-29 18:00:00
978054,2021-11-29 18:03:19,2.10,3.81,4.66,18123.0,-66.0,47.0,38.0,3.81,2021-11-29 18:00:00
978055,2021-11-29 18:05:19,2.31,3.81,3.81,18125.0,-66.0,47.0,38.0,3.81,2021-11-29 18:00:00
978056,2021-11-29 18:07:19,2.36,4.11,4.18,18127.0,-67.0,47.0,39.0,4.11,2021-11-29 18:00:00
978057,2021-11-29 18:09:19,1.40,2.96,3.70,18129.0,-66.0,47.0,39.0,2.96,2021-11-29 18:15:00
...,...,...,...,...,...,...,...,...,...,...
931261,2021-08-30 17:36:56,5.02,7.78,8.00,8651.0,-61.0,87.0,51.0,7.78,2021-08-30 17:30:00
931262,2021-08-30 17:38:56,5.14,6.71,6.71,8653.0,-61.0,87.0,51.0,6.71,2021-08-30 17:45:00
931263,2021-08-30 17:40:56,4.70,6.34,6.34,8655.0,-61.0,87.0,51.0,6.34,2021-08-30 17:45:00


In [16]:
df_Urbana['new_date'] = df_Urbana.apply(lambda time: each_fifteen(time['created_at']),axis=1)
df_Urbana

,created_at,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,new_date
entry_id,,,,,,,,,,
194084,2021-11-29 18:00:45,6.02,6.93,7.28,316.0,-58.0,44.0,45.0,6.93,2021-11-29 18:00:00
194085,2021-11-29 18:02:45,6.86,7.68,7.70,318.0,-58.0,44.0,46.0,7.68,2021-11-29 18:00:00
194086,2021-11-29 18:04:45,6.68,7.43,7.91,320.0,-59.0,44.0,46.0,7.43,2021-11-29 18:00:00
194087,2021-11-29 18:06:45,6.56,7.88,8.27,322.0,-59.0,44.0,46.0,7.88,2021-11-29 18:00:00
194088,2021-11-29 18:08:45,6.79,8.16,8.83,324.0,-59.0,44.0,46.0,8.16,2021-11-29 18:15:00
...,...,...,...,...,...,...,...,...,...,...
128716,2021-08-30 18:50:11,3.37,4.39,4.59,15779.0,-58.0,86.0,54.0,4.39,2021-08-30 18:45:00
128717,2021-08-30 18:52:11,3.45,4.34,4.34,15781.0,-56.0,86.0,54.0,4.34,2021-08-30 18:45:00
128718,2021-08-30 18:54:11,4.00,5.05,5.57,15783.0,-55.0,85.0,55.0,5.05,2021-08-30 19:00:00


In [17]:
df_UrbanaOnHolmes['new_date'] = df_UrbanaOnHolmes.apply(lambda time: each_fifteen(time['created_at']),axis=1)
df_UrbanaOnHolmes

,created_at,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,new_date
entry_id,,,,,,,,,,
94763,2021-12-05 18:11:41,19.62,31.23,34.21,10.0,-66.0,56.0,75.0,30.29,2021-12-05 18:15:00
94764,2021-12-05 18:13:41,19.92,31.98,34.02,12.0,-66.0,56.0,75.0,30.84,2021-12-05 18:15:00
94765,2021-12-05 18:15:41,19.80,33.96,35.29,14.0,-70.0,56.0,75.0,32.21,2021-12-05 18:15:00
94766,2021-12-05 18:17:41,20.14,31.89,34.39,16.0,-68.0,56.0,75.0,30.89,2021-12-05 18:15:00
94767,2021-12-05 18:19:41,20.43,33.64,35.89,18.0,-68.0,56.0,75.0,31.70,2021-12-05 18:15:00
...,...,...,...,...,...,...,...,...,...,...
30277,2021-08-30 18:50:16,2.69,4.24,4.24,31567.0,-65.0,83.0,59.0,4.24,2021-08-30 18:45:00
30278,2021-08-30 18:52:16,2.60,4.14,4.14,31569.0,-64.0,83.0,60.0,4.14,2021-08-30 18:45:00
30279,2021-08-30 18:54:16,2.62,4.14,4.30,31571.0,-66.0,83.0,59.0,4.14,2021-08-30 19:00:00


In [18]:
df_WarnowChacko_date = df_WarnowChacko.drop(columns = ['created_at'])
df_WarnowChacko_date.sort_values('new_date', ascending=True)

,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,new_date
entry_id,,,,,,,,,
926266,11.79,18.14,19.02,7673.0,-65.0,94.0,45.0,18.14,2021-08-23 19:00:00
926265,12.02,17.07,18.05,7671.0,-65.0,94.0,45.0,17.07,2021-08-23 19:00:00
926267,11.85,17.62,18.69,7675.0,-69.0,94.0,45.0,17.62,2021-08-23 19:00:00
926264,12.54,19.31,19.76,7669.0,-71.0,94.0,45.0,19.31,2021-08-23 19:00:00
926268,11.72,17.15,18.07,7677.0,-67.0,94.0,45.0,17.15,2021-08-23 19:15:00
...,...,...,...,...,...,...,...,...,...
982989,0.00,0.00,0.00,256.0,-64.0,40.0,25.0,0.00,2021-12-06 17:45:00
982991,0.00,0.25,0.25,260.0,-66.0,40.0,24.0,0.25,2021-12-06 17:45:00
982994,0.00,0.00,0.00,266.0,-66.0,40.0,24.0,0.00,2021-12-06 18:00:00


In [19]:
df_Urbana_date = df_Urbana.drop(columns = ['created_at'])
df_Urbana_date.sort_values('new_date', ascending=True)

,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,new_date
entry_id,,,,,,,,,
123682,12.32,15.88,16.96,5709.0,-60.0,94.0,45.0,15.88,2021-08-23 19:00:00
123683,12.93,18.24,18.83,5711.0,-56.0,94.0,44.0,18.24,2021-08-23 19:00:00
123684,12.15,16.14,16.46,5713.0,-59.0,93.0,44.0,16.14,2021-08-23 19:00:00
123685,12.57,15.69,16.34,5715.0,-58.0,93.0,45.0,15.69,2021-08-23 19:00:00
123686,12.87,17.09,17.65,5717.0,-58.0,93.0,43.0,17.09,2021-08-23 19:15:00
...,...,...,...,...,...,...,...,...,...
199018,0.24,0.59,1.02,258.0,-57.0,36.0,31.0,0.59,2021-12-06 17:45:00
199019,0.55,0.95,1.61,260.0,-56.0,36.0,32.0,0.95,2021-12-06 17:45:00
199022,0.34,1.00,1.71,266.0,-55.0,35.0,32.0,1.00,2021-12-06 18:00:00


In [20]:
df_UrbanaOnHolmes_date = df_UrbanaOnHolmes.drop(columns = ['created_at'])
df_UrbanaOnHolmes_date.sort_values('new_date', ascending=True)

,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,new_date
entry_id,,,,,,,,,
25243,12.04,18.18,18.46,21498.0,-66.0,91.0,50.0,18.18,2021-08-23 19:00:00
25244,11.61,18.23,18.30,21500.0,-64.0,91.0,50.0,18.23,2021-08-23 19:00:00
25245,12.64,20.76,20.91,21502.0,-66.0,91.0,51.0,20.76,2021-08-23 19:00:00
25246,13.46,19.07,19.71,21504.0,-67.0,91.0,51.0,19.07,2021-08-23 19:00:00
25247,12.78,19.12,19.31,21506.0,-68.0,91.0,51.0,19.12,2021-08-23 19:15:00
...,...,...,...,...,...,...,...,...,...
95468,0.82,1.67,2.11,237.0,-68.0,32.0,32.0,1.67,2021-12-06 17:45:00
95472,1.20,1.86,2.30,245.0,-69.0,33.0,32.0,1.86,2021-12-06 18:00:00
95473,1.86,3.14,3.84,247.0,-71.0,33.0,32.0,3.14,2021-12-06 18:00:00


In [21]:
df_WarnowChacko_date_group = df_WarnowChacko_date.groupby('new_date').agg('mean').reset_index()
df_WarnowChacko_date_group

,new_date,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3
0,2021-08-23 19:00:00,12.050000,18.035000,18.880000,7672.0,-67.500000,94.000000,45.000000,18.035000
1,2021-08-23 19:15:00,12.226250,17.927500,18.696250,7684.0,-67.250000,94.000000,45.000000,17.927500
2,2021-08-23 19:30:00,11.734286,17.234286,17.677143,7699.0,-66.714286,92.571429,45.000000,17.234286
3,2021-08-23 19:45:00,11.942500,17.366250,18.236250,7714.0,-68.125000,93.000000,45.000000,17.366250
4,2021-08-23 20:00:00,13.081429,19.161429,19.652857,7729.0,-68.714286,93.000000,45.142857,19.161429
...,...,...,...,...,...,...,...,...,...
7565,2021-12-06 17:00:00,0.000000,0.037143,0.100000,208.0,-62.714286,41.000000,25.285714,0.037143
7566,2021-12-06 17:15:00,0.138750,0.446250,0.623750,223.0,-62.375000,41.000000,25.000000,0.446250
7567,2021-12-06 17:30:00,0.015714,0.040000,0.152857,238.0,-65.285714,40.857143,25.142857,0.040000
7568,2021-12-06 17:45:00,0.000000,0.046250,0.046250,253.0,-65.000000,40.000000,24.750000,0.046250


In [22]:
df_Urbana_date_group = df_Urbana_date.groupby('new_date').agg('mean').reset_index()
df_Urbana_date_group[df_Urbana_date_group['PM2.5 (CF=1) ug/m3']== 7.6]

,new_date,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3
903,2021-09-02 04:45:00,5.43125,7.6,8.1275,19254.0,-55.625,68.0,61.0,7.6
8338,2021-11-18 18:15:00,5.44250,7.6,8.4450,2299.0,-54.750,41.0,42.0,7.6


In [23]:
df_UrbanaOnHolmes_date_group = df_UrbanaOnHolmes_date.groupby('new_date').agg('mean').reset_index()
df_UrbanaOnHolmes_date_group

,new_date,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3
0,2021-08-23 19:00:00,12.437500,19.060000,19.345000,21501.0,-65.750000,91.000000,50.500000,19.060000
1,2021-08-23 19:15:00,12.312857,18.508571,19.091429,21512.0,-65.428571,91.000000,50.285714,18.508571
2,2021-08-23 19:30:00,12.196250,18.246250,18.832500,21527.0,-63.875000,90.250000,50.875000,18.246250
3,2021-08-23 19:45:00,12.372857,18.391429,19.118571,21542.0,-64.571429,90.000000,52.142857,18.391429
4,2021-08-23 20:00:00,12.816250,19.023750,20.082500,21557.0,-64.250000,89.125000,53.500000,19.023750
...,...,...,...,...,...,...,...,...,...
9362,2021-12-06 17:00:00,0.313750,0.883750,1.187500,192.0,-67.500000,33.875000,31.000000,0.883750
9363,2021-12-06 17:15:00,0.264286,0.895714,1.085714,207.0,-69.428571,33.714286,31.571429,0.895714
9364,2021-12-06 17:30:00,0.335000,0.981250,1.270000,222.0,-67.750000,33.125000,31.875000,0.981250
9365,2021-12-06 17:45:00,0.745714,1.545714,1.877143,237.0,-68.857143,32.428571,32.000000,1.545714


In [24]:
df_WarnowChacko_date_group.to_feather('export_WarnowChacko_feather')
df_Urbana_date_group.to_feather('export_Urbana_feather')
df_UrbanaOnHolmes_date_group.to_feather('export_UrbanaOnHolmes_feather')

In [25]:
df_WarnowChacko_date_group.append(df_Urbana_date_group)

,new_date,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3
0,2021-08-23 19:00:00,12.050000,18.035000,18.880000,7672.0,-67.500000,94.000000,45.000000,18.035000
1,2021-08-23 19:15:00,12.226250,17.927500,18.696250,7684.0,-67.250000,94.000000,45.000000,17.927500
2,2021-08-23 19:30:00,11.734286,17.234286,17.677143,7699.0,-66.714286,92.571429,45.000000,17.234286
3,2021-08-23 19:45:00,11.942500,17.366250,18.236250,7714.0,-68.125000,93.000000,45.000000,17.366250
4,2021-08-23 20:00:00,13.081429,19.161429,19.652857,7729.0,-68.714286,93.000000,45.142857,19.161429
...,...,...,...,...,...,...,...,...,...
10047,2021-12-06 17:00:00,0.034286,0.408571,0.734286,208.0,-56.714286,36.000000,30.142857,0.408571
10048,2021-12-06 17:15:00,0.086250,0.553750,0.763750,223.0,-57.125000,36.000000,30.875000,0.553750
10049,2021-12-06 17:30:00,0.268571,0.608571,0.915714,238.0,-56.857143,36.000000,31.000000,0.608571
10050,2021-12-06 17:45:00,0.313750,0.813750,1.206250,253.0,-57.125000,36.000000,31.125000,0.813750


In [26]:
columns = ['new_date', 'PM1.0 (CF=1) ug/m3', 'PM2.5 (CF=1) ug/m3', 'PM10.0 (CF=1) ug/m3',
          'UptimeMinutes', 'ADC', 'Temperature_F', 'Humidity_%', 'PM2.5 (CF=ATM) ug/m3', 'name']
df_sensor = pd.DataFrame(columns = columns)
df_sensor

,new_date,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,name


In [27]:
df_sensor.append(df_WarnowChacko_date_group)

,new_date,PM1.0 (CF=1) ug/m3,PM2.5 (CF=1) ug/m3,PM10.0 (CF=1) ug/m3,UptimeMinutes,ADC,Temperature_F,Humidity_%,PM2.5 (CF=ATM) ug/m3,name
0,2021-08-23 19:00:00,12.050000,18.035000,18.880000,7672.0,-67.500000,94.000000,45.000000,18.035000,NaN
1,2021-08-23 19:15:00,12.226250,17.927500,18.696250,7684.0,-67.250000,94.000000,45.000000,17.927500,NaN
2,2021-08-23 19:30:00,11.734286,17.234286,17.677143,7699.0,-66.714286,92.571429,45.000000,17.234286,NaN
3,2021-08-23 19:45:00,11.942500,17.366250,18.236250,7714.0,-68.125000,93.000000,45.000000,17.366250,NaN
4,2021-08-23 20:00:00,13.081429,19.161429,19.652857,7729.0,-68.714286,93.000000,45.142857,19.161429,NaN
...,...,...,...,...,...,...,...,...,...,...
7565,2021-12-06 17:00:00,0.000000,0.037143,0.100000,208.0,-62.714286,41.000000,25.285714,0.037143,NaN
7566,2021-12-06 17:15:00,0.138750,0.446250,0.623750,223.0,-62.375000,41.000000,25.000000,0.446250,NaN
7567,2021-12-06 17:30:00,0.015714,0.040000,0.152857,238.0,-65.285714,40.857143,25.142857,0.040000,NaN
7568,2021-12-06 17:45:00,0.000000,0.046250,0.046250,253.0,-65.000000,40.000000,24.750000,0.046250,NaN


# To do

1. Make the functions to help us get the feather file with senso's ID